In [ ]:
{-# LANGUAGE FunctionalDependencies #-}
{-# LANGUAGE MonomorphismRestriction #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE ConstraintKinds #-}
{-# LANGUAGE TemplateHaskell #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE BangPatterns #-}

import Control.Arrow
import Control.Monad.State
import Control.Monad.Trans.Class
import Numeric.LinearAlgebra
import Numeric.LinearAlgebra.Data
import Data.List.Split
import Data.List
import Numeric
import Prelude hiding ((<>))
import Network.HTTP.Client
import Network.HTTP.Types.Status (statusCode)
import qualified Codec.Compression.GZip as GZip
import System.Directory
import System.FilePath.Posix
import System.Random
import qualified Data.ByteString.Lazy  as BS
import qualified Data.ByteString as BSS
import Data.Binary as B
import Data.Binary.Get.Internal as BGI
import Data.Time.Format
import Data.Time.LocalTime
import Debug.Trace

\begin{eqnarray}
Sigmoid \nonumber \\
  h(x) &=& \frac
  {1}
  {1 + \exp(-x)} \\
\nonumber \\
ReLU \nonumber \\
  h(x) &=& \begin{cases}
  x & (x > 0) \\
  0 & (x \le 0)
  \end{cases}
\end{eqnarray}

In [ ]:
sigmoid :: (Floating a) => a -> a
sigmoid a = fromIntegral 1 / (fromIntegral 1 + exp (-a))

sigmoidm :: (Floating a, Container Matrix a) => Matrix a -> Matrix a
sigmoidm = cmap sigmoid

sigmoidBackward :: (Floating a, Num (Vector a), Container Matrix a) => Matrix a -> Matrix a -> Matrix a
sigmoidBackward y d = d * (cmap (\a -> fromIntegral 1 - a) y) * y

relu :: (Ord a, Num a) => a -> a
relu = max $ fromIntegral 0

relum :: (Ord a, Num a, Container Matrix a) => Matrix a -> Matrix a
relum = cmap relu

relumBackward :: (Ord a, Num a, Num (Matrix a), Container Vector a, Container Matrix a) => Matrix a -> Matrix a -> Matrix a
relumBackward x d = d * mask
    where mask = cmap (\a -> fromIntegral $ if (0 < a) then 1 else 0) x

\begin{eqnarray}
Cross Entropy \nonumber \\
  L &=& -\displaystyle \sum_{i=1}^{n}t_i \log (y_i + d) \\
  t &:& 教師ラベル \nonumber \\
  d &:& 微小値 \nonumber \\
 \nonumber \\
Softmax \nonumber \\
  y_k &=& \frac
  {\exp(a_k - \hat{a})}
  {\displaystyle \sum_{i=1}^{n}\exp(a_i - \hat{a})} \\
  \hat{a} &=& \max \{ a_{1...n} \} \nonumber
\end{eqnarray}


In [ ]:
softmax :: (Floating a, Container Vector a) => Vector a -> Vector a
softmax v = cmap (/s) v'
    where
        m = maxElement v
        v' = cmap (\a -> exp (a - m)) v
        s = sumElements v'

softmaxm :: (Floating a, Container Vector a) => Matrix a -> Matrix a
softmaxm m = fromRows $ map softmax $ toRows m

crossEntropy :: (Floating a, Num (Vector a), Container Vector a) => Vector a -> Vector a -> a
crossEntropy t y = -(sumElements $ cmap log (y + d) * t)
    where d = 1e-10

crossEntropym :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> a
crossEntropym t m = sum vs / batchSize
    where 
        vs = uncurry crossEntropy `map` (toRows t `zip` toRows m)
        batchSize = fromIntegral $ rows t

softmaxWithCross :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> a
softmaxWithCross t = crossEntropym t . softmaxm

softmaxWithCrossBackward :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> Matrix a
softmaxWithCrossBackward t y = (y - t) / batchSize
    where batchSize = fromIntegral $ rows t

In [ ]:
affinem :: (Floating a, Numeric a, Num (Vector a)) => Matrix a -> Vector a -> Matrix a -> Matrix a
affinem w b x = x <> w + b'
    where b' = fromRows $ replicate (rows x) b

affinemBackward :: (Floating a, Numeric a) => Matrix a -> Matrix a -> Matrix a -> (Matrix a, Matrix a, Vector a)
affinemBackward w x d = (dx, dw, db)
    where
        dx = d <> tr w
        dw = tr x <> d
        db = fromList $ sumElements `map` toColumns d

In [ ]:
type Weight a = Matrix a
type Bias a = Vector a
type SignalX a = Matrix a
type SignalY a = Matrix a
type Diff a = Matrix a
type TeacherBatch a = Matrix a
type InputBatch a = Matrix a
newtype TrainBatch a = TrainBatch (TeacherBatch a, InputBatch a) deriving (Show)

In [ ]:
data ForwardLayer a =
    AffineForward (Weight a) (Bias a)
  | SigmoidForward
  | ReLUForward
  | JoinedForwardLayer (ForwardLayer a) (ForwardLayer a)
  deriving (Show)

infixl 4 ~>
(~>) :: ForwardLayer a -> ForwardLayer a -> ForwardLayer a
a ~> (JoinedForwardLayer x y) = (a ~> x) ~> y
a ~> b = JoinedForwardLayer a b

data OutputLayer a = SoftmaxWithCrossForward
  deriving (Show)

data ForwardNN a = ForwardNN (ForwardLayer a) (OutputLayer a)
  deriving (Show)

data BackwardLayer a = 
    AffineBackward (Weight a) (Bias a) (SignalX a)
  | SigmoidBackward (SignalY a)
  | ReLUBackward (SignalX a)
  | JoinedBackwardLayer (BackwardLayer a) (BackwardLayer a)
  deriving (Show)

infixr 4 <~
(<~) :: BackwardLayer a -> BackwardLayer a -> BackwardLayer a
(JoinedBackwardLayer x y) <~ b = x <~ (y <~ b)
a <~ b = JoinedBackwardLayer a b

data BackputLayer a = SoftmaxWithCrossBackward (TeacherBatch a) (SignalY a)

data BackwardNN a = BackwardNN (BackwardLayer a) (BackputLayer a)

type NElement a = (Ord a, Floating a, Numeric a, Num (Vector a), Show a)

In [ ]:
forward :: NElement a => ForwardLayer a -> SignalX a -> (BackwardLayer a, SignalY a)
forward (AffineForward w b) x = (AffineBackward w b x, affinem w b x)
forward SigmoidForward x = let y = sigmoidm x in (SigmoidBackward y, y)
forward ReLUForward x = (ReLUBackward x, relum x)
forward (JoinedForwardLayer a b) x0 = (a' <~ b', x2)
    where
        (a', x1) = forward a x0
        (b', x2) = forward b x1

backward :: NElement a => BackwardLayer a -> Diff a -> (ForwardLayer a, Diff a)
backward (AffineBackward w b x) d =  (AffineForward (w + w') (b + b'), x')
    where (x', w', b') = affinemBackward w x d
backward (SigmoidBackward y) d = (SigmoidForward, sigmoidBackward y d)
backward (ReLUBackward x) d = (ReLUForward, relumBackward x d)
backward (JoinedBackwardLayer a b) d0 = (a' ~> b', d2)
    where
        (b', d1) = backward b d0
        (a', d2) = backward a d1

output :: NElement a => OutputLayer a -> TeacherBatch a -> SignalY a -> (BackputLayer a, a)
output SoftmaxWithCrossForward t y = (SoftmaxWithCrossBackward t y, softmaxWithCross t y)

backput :: NElement a => BackputLayer a -> (OutputLayer a, Diff a)
backput (SoftmaxWithCrossBackward t y) = (SoftmaxWithCrossForward, softmaxWithCrossBackward t y)

In [ ]:
learnForward :: NElement a => ForwardNN a -> TrainBatch a -> (BackwardNN a, a)
learnForward (ForwardNN layers loss) (TrainBatch (t, x)) = result `seq` (BackwardNN layers' loss', result)
    where
        (layers', y) = forward layers x
        (loss', result) = output loss t y

learnBackward :: NElement a => BackwardNN a -> ForwardNN a
learnBackward  (BackwardNN layers loss) = ForwardNN layers' loss'
    where
        (loss', d) = backput loss
        (layers', _) = backward layers d

learn :: NElement a => ForwardNN a -> TrainBatch a -> (ForwardNN a, a)
learn a = first learnBackward . learnForward a

learnAll :: NElement a => ForwardNN a -> [TrainBatch a] -> (ForwardNN a, [a])
learnAll origin = foldr f (origin, [])
    where f batch (a, results) = trace ("Learning iterate: " ++ show (length results + 1)) $ a `seq` second (:results) $ learn a batch

predict :: NElement a => ForwardLayer a -> Vector a -> Int
predict layers = maxIndex . flatten . snd . (forward layers) . asRow

evaluate :: NElement a => ForwardLayer a -> [(Int, Vector a)] -> Double
evaluate layers samples = fromIntegral nOk / fromIntegral (length samples)
    where
        nOk = length $ filter (\(a, b) -> a == b) results
        results = map (second $ predict layers) samples

In [ ]:
newtype MnistLabels = MnistLabels [Word8] deriving (Show)
newtype MnistImages = MnistImages [Matrix Z] deriving (Show)
newtype MnistData = MnistData [(Word8, Matrix Z)] deriving (Show)

zipMnist :: MnistLabels -> MnistImages -> MnistData
zipMnist (MnistLabels labels) (MnistImages images) = MnistData $ labels `zip` images

markMinistLabels = 2049
markMnistImages = 2051

putAsWord32 :: Integral a => a -> Put
putAsWord32 a = B.put (fromIntegral a :: Word32)

getAsIntegral :: Integral a => Get a
getAsIntegral = fromIntegral <$> (B.get :: Get Word32)

instance B.Binary MnistLabels where
    put (MnistLabels labels) = do
        B.put markMinistLabels
        putAsWord32 $ length labels
        B.put $ BS.pack labels

    get = do
        mark <- getAsIntegral
        guard $ mark == markMinistLabels
        size <- getAsIntegral
        let total = trace ("Reading labels: " ++ show size) size
        labels <- BGI.readN total BSS.unpack
        return $ MnistLabels $ labels

instance B.Binary MnistImages where
    put (MnistImages (images @ (hm:_))) = do
        putAsWord32 markMnistImages
        putAsWord32 $ length images
        putAsWord32 $ rows hm
        putAsWord32 $ cols hm
        B.put $ (BS.pack . map fromIntegral . concat . concat . map toLists) images

    get = do
        mark <- getAsIntegral
        guard $ mark == markMnistImages
        size <- getAsIntegral
        nRows <- getAsIntegral
        nCols <- getAsIntegral
        let total = trace ("Reading images: " ++ show size ++ " of " ++ show nRows ++ "x" ++ show nCols) size * nRows * nCols
        zs <- map fromIntegral <$> BGI.readN total BSS.unpack
        let images = map (\a -> enRows >< nCols $! a) $ chunksOf (nRows * nCols) zs
        return $ MnistImages $! images

In [ ]:
timestamp msg = do
    t <- getZonedTime
    let stamp = formatTime defaultTimeLocale "[%Y/%m/%d %H:%M:%S] " t
    print $ stamp ++ msg

download :: Manager -> String -> IO BS.ByteString
download manager url = do
   request <- parseRequest url
   timestamp $ "Downloading " ++ url
   response <- httpLbs request manager
   return $ responseBody response

saveMnist :: FilePath -> String -> [String] -> Manager -> IO [FilePath]
saveMnist rootDir urlBase filenames manager = do
    isDir <- doesDirectoryExist rootDir
    _ <- if isDir then return () else createDirectory rootDir
    mapM saveFile filenames
    where
        saveFile filename = do
            let url = urlBase ++ filename
            let file = rootDir </> filename
            timestamp $ "Checking file " ++ file
            e <- doesFileExist file
            if e then return () else download manager url >>= (BS.writeFile file)
            return file

readMnist :: FilePath -> IO (Either MnistLabels MnistImages)
readMnist file = do
    timestamp $ "decoding " ++ file ++ " ..."
    bs <- GZip.decompress <$> BS.readFile file
    let a = left (const $ B.decode bs) $ right (\(_, _, a) -> a) $ B.decodeOrFail bs
    timestamp $ "decoded " ++ file
    return a

In [ ]:
shuffleList :: Show a => [a] -> IO [a]
shuffleList src =  doShuffle [] src <$> newStdGen
    where
        doShuffle rs [] g = rs
        doShuffle rs (x:xs) g = doShuffle (ys ++ (trace "Picking element" x):zs) xs g'
            where
                (i, g') = randomR (0, length rs - 1) g
                (ys, zs) = splitAt i rs

In [ ]:
normalAffine :: Int -> Int -> IO (ForwardLayer R)
normalAffine nIn nOut = do
        weights <- rand nIn nOut
        bias <- flatten <$> rand nOut 1
        return $ (AffineForward weights bias) ~> ReLUForward

initNN :: IO (ForwardNN R)
initNN = do
    affine1 <- normalAffine (28 * 28) 100
    affine2 <- normalAffine 100 10
    return $ ForwardNN (affine1 ~> affine2) SoftmaxWithCrossForward

convertTrains :: Int -> MnistData -> [TrainBatch R]
convertTrains batchSize (MnistData src) = map (mkTrainer . unzip) $ chunksOf batchSize $! vectors
    where
        vectors = map (first (\a -> hotone 10 $! a) . second (\a -> flatten $! a)) src
        mkTrainer (a, b) = TrainBatch (fromRows a, 255 / (fromZ $ fromRows b))

hotone :: (Integral v, NElement a) => Int -> v -> (Vector a)
hotone n v = fromList $ map fromIntegral list
    where
        list = replicate i 0 ++ 1 : replicate (n - i - 1) 0
        i = fromIntegral v

convertTests :: MnistData -> [(Int, Vector R)]
convertTests (MnistData src) = map (first fromIntegral . second ((255/) . flatten . fromZ)) src

shuffleMnist :: MnistData -> IO MnistData
shuffleMnist (MnistData list) = MnistData <$> return list

trainingSimple :: ForwardNN R -> MnistData -> MnistData -> IO Double
trainingSimple nn trainData testData = do
    trainBatches <- convertTrains 100 <$> shuffleMnist trainData
    let (ForwardNN layers _, _) = learnAll nn trainBatches
    let rate = evaluate layers $ convertTests testData
    return rate

In [ ]:
newManager defaultManagerSettings >>= saveMnist "mnist" "http://yann.lecun.com/exdb/mnist/" [
    "train-images-idx3-ubyte.gz"
  , "train-labels-idx1-ubyte.gz"
  , "t10k-images-idx3-ubyte.gz"
  , "t10k-labels-idx1-ubyte.gz"
  ] >>= mapM readMnist >>= \ms -> do
      let [trainers, tests] = map (\[Right a, Left b] -> b `zipMnist` a) $ chunksOf 2 ms
      origin <- initNN
      result <- trainingSimple origin trainers tests
      timestamp $ "result=" ++ show (result * 100) ++ "%"
      return result